# Drums drums drums drums drums

Philly Joe Jones - "Billy Boy", from Miles Davis' *Milestones* (1958)

Transcribed by yours truly

In [1]:
import os

import mido
import music21

import jazzaiexperiments

Using TensorFlow backend.


In [2]:
# Comping
# tune_name = "billyboy_comping"
# input_filepath = "../data/midi/mine/billyboy/PhillyJoeJones_BillyBoy_Comping_Processed77.mid"

# Solo
tune_name = "billyboy_solo"
input_filepath = "../data/midi/mine/billyboy/PhillyJoeJones_BillyBoy_Solo_Processed77.mid"

Let's see if it works just training it in `single_melody` mode:

In [3]:
# model, note_events, input_filepath = jazzaiexperiments.lstm.train_on_midi_input(tune_name,
#                                                                                 mode="single_melody",
#                                                                                 input_filepath=input_filepath,
#                                                                                 num_epochs=1)
# model.summary()

In [4]:
# model = jazzaiexperiments.lstm.load_model_weights(model, "../data/models/weights_billyboy_solo_20170706143300474703_99_0.6481.hdf5")
# model.summary()

In [5]:
# print("{} note events".format(len(note_events)))
# print("{} unique note events".format(len(jazzaiexperiments.midi.create_note_set(note_events))))


# # jazzaiexperiments.midi.create_note_set(note_events)[:10]
# [jazzaiexperiments.midi.note_event_to_dict(evt, "single_melody") for evt in jazzaiexperiments.midi.create_note_set(note_events)[:5]]

In [6]:
# notes_out = jazzaiexperiments.lstm.generate_midi_output(model, note_events,
#                                                         num_notes_to_generate=100,
#                                                         time_multiplier=4,
#                                                         random_seed=False,
#                                                         add_seed_to_output=True,
#                                                         tune_name=tune_name,
#                                                         midi_source_filepath=input_filepath,
#                                                         data_dir="../data/output")
# notes_out[:20]

In [7]:
# jazzaiexperiments.midi.write_file(note_events, os.path.join("../data/output/", "{}_original.mid".format(tune_name)),
#                                   mode="single_melody",
#                                   time_multiplier=5,
#                                   midi_source_filepath=input_filepath)


... Not really. Among the issues are lack of **polyphony**, and the **timing** (which is tied to the `time_multiplier`, which has to be manually set every time, which is bad)

### Fixing polyphony

One problem seems to be that the `single_melody` mode can't really handle notes that occur simultaneously. Thus we have to build in some kind of polyphony handling.

Functions to update:
- midi.extract_note_pairs
- midi.normalize_velocities
- midi.create_note_events
- lstm.train_on_midi_input

In [8]:
from jazzaiexperiments import *

In [9]:
midi_file = mido.MidiFile(input_filepath)
midi_track = midi_file.tracks[0]
[msg for msg in midi_track[:100]]

[<meta message track_name name='\x00' time=0>,
 <meta message time_signature numerator=4 denominator=4 clocks_per_click=36 notated_32nd_notes_per_beat=8 time=0>,
 <meta message time_signature numerator=4 denominator=4 clocks_per_click=36 notated_32nd_notes_per_beat=8 time=0>,
 <message control_change channel=0 control=91 value=19 time=0>,
 <message control_change channel=0 control=91 value=0 time=0>,
 <message control_change channel=0 control=91 value=0 time=768>,
 <message control_change channel=0 control=91 value=19 time=0>,
 <message note_on channel=0 note=51 velocity=66 time=0>,
 <message note_off channel=0 note=51 velocity=0 time=48>,
 <message note_on channel=0 note=38 velocity=57 time=16>,
 <message note_on channel=0 note=44 velocity=74 time=32>,
 <message note_on channel=0 note=51 velocity=74 time=0>,
 <message note_off channel=0 note=38 velocity=0 time=16>,
 <message note_off channel=0 note=44 velocity=0 time=32>,
 <message note_off channel=0 note=51 velocity=0 time=0>,
 <mess

In [10]:
# From lstm.train_on_midi_input()
midi_track = midi.load_melody_from_file(input_filepath)
note_pairs = midi.extract_note_pairs(midi_track)
note_pairs = midi.normalize_velocities(note_pairs, interval=10)
print("Track has {} note messages; {} note pairs created".format(len(midi.extract_note_messages(midi_track)), len(note_pairs)))
print("... which means we're missing {} notes".format((len(midi.extract_note_messages(midi_track))/2) - len(note_pairs)))

Track has 2086 note messages; 1024 note pairs created
... which means we're missing 19.0 notes


So let's redefine our `extract_note_pairs` function:

In [11]:
note_pairs[0][-1]

<message note_off channel=0 note=51 velocity=0 time=48>

In [12]:
def extract_note_pairs(track, enable_polyphony=False):
    """Extract note on/off pairs from a MIDI track.
    
    If polyphony is enabled, each pair will actually consist of a list
    of note ons and a list of note offs in the form (noteons, noteoffs).
    """
    notes = midi.extract_note_messages(track)
    note_pairs = []
    if enable_polyphony:
        note_pair = ([], [])
        for note in notes:
            if note.type == "note_on":
                note_ons = note_pair[0]
                note_offs = note_pair[1]
                if len(note_ons) < 1 or note.time == 0:
                    note_ons.append(note)
                    note_pair = (note_ons, note_offs)
                else:
                    for existing_note in note_ons:
                        note_off = mido.Message("note_off",
                                                channel=existing_note.channel,
                                                note=existing_note.note,
                                                velocity=existing_note.velocity,
                                                time=note.time)
                        note_offs.append(note_off)
                    note_pair = (note_ons, note_offs)
                    note_pairs.append(note_pair)
                    note_pair = ([], [])
            elif note.type == "note_off":
                note_ons = note_pair[0]
                note_offs = note_pair[1]
                if len(note_ons) > 0:
                    for existing_note in note_ons:
                        note_off = mido.Message("note_off",
                                                channel=existing_note.channel,
                                                note=existing_note.note,
                                                velocity=existing_note.velocity,
                                                time=note.time)
                        note_offs.append(note_off)
                    note_pair = (note_ons, note_offs)
                    note_pairs.append(note_pair)
                    note_pair = ([], [])
    else:
        note_pairs = [(notes[i], notes[i + 1]) for i, _ in enumerate(notes[:-1])
                      if notes[i].type == "note_on" and
                      notes[i + 1].type == "note_off" and
                      notes[i].note == notes[i + 1].note]
    return note_pairs

In [13]:
def normalize_velocities(note_pairs, interval=10):
    """Normalize note velocities."""
    for i, note_pair in enumerate(note_pairs):
        note_ons, note_offs = note_pair
        
        if type(note_ons) is list:
            for note_on in note_ons:
                note_on.velocity = note_on.velocity - (note_on.velocity % interval)
        else:
            note_on = note_ons
            note_on.velocity = note_on.velocity - (note_on.velocity % interval)
        
        note_pair = (note_ons, note_offs)
        note_pairs[i] = note_pair
                
    return note_pairs

In [14]:
midi_track = midi.load_melody_from_file(input_filepath)
note_pairs = extract_note_pairs(midi_track, enable_polyphony=True)
note_pairs = normalize_velocities(note_pairs, interval=10)
print(len([(len(note_pair[0]), len(note_pair[1])) for note_pair in note_pairs if len(note_pair[0]) > 1]))
[(len(note_pair[0]), len(note_pair[1])) for note_pair in note_pairs if len(note_pair[0]) > 1][:10]

81


[(2, 2),
 (2, 2),
 (2, 2),
 (2, 2),
 (2, 2),
 (2, 2),
 (3, 3),
 (2, 2),
 (2, 2),
 (2, 2)]

In [15]:
[(i, len(note_pair[0]), len(note_pair[1])) for i, note_pair in enumerate(note_pairs)][:20]

[(0, 1, 1),
 (1, 1, 1),
 (2, 1, 1),
 (3, 1, 1),
 (4, 1, 1),
 (5, 1, 1),
 (6, 1, 1),
 (7, 1, 1),
 (8, 1, 1),
 (9, 1, 1),
 (10, 1, 1),
 (11, 1, 1),
 (12, 1, 1),
 (13, 1, 1),
 (14, 1, 1),
 (15, 2, 2),
 (16, 1, 1),
 (17, 1, 1),
 (18, 1, 1),
 (19, 2, 2)]

In [16]:
len(note_pairs)

803

In [17]:
note_pairs[:10]

[([<message note_on channel=0 note=51 velocity=60 time=0>],
  [<message note_off channel=0 note=51 velocity=66 time=48>]),
 ([<message note_on channel=0 note=38 velocity=50 time=16>],
  [<message note_off channel=0 note=38 velocity=57 time=32>]),
 ([<message note_on channel=0 note=51 velocity=70 time=0>],
  [<message note_off channel=0 note=51 velocity=74 time=16>]),
 ([<message note_on channel=0 note=51 velocity=70 time=16>],
  [<message note_off channel=0 note=51 velocity=71 time=32>]),
 ([<message note_on channel=0 note=51 velocity=60 time=0>],
  [<message note_off channel=0 note=51 velocity=62 time=96>]),
 ([<message note_on channel=0 note=44 velocity=60 time=0>],
  [<message note_off channel=0 note=44 velocity=66 time=0>]),
 ([<message note_on channel=0 note=51 velocity=60 time=0>],
  [<message note_off channel=0 note=51 velocity=66 time=48>]),
 ([<message note_on channel=0 note=51 velocity=60 time=16>],
  [<message note_off channel=0 note=51 velocity=66 time=32>]),
 ([<message no

Alright, let's see if that works with the rest of the input pipeline:

In [18]:
def create_note_events(note_pairs, mode="single_melody", chords=[]):
    """Create note events from note pairs.

    This is the base data structure for note manipulation.
    """
    note_events = []
    if mode == "single_melody":
        note_events = [(note_on.note, note_on.velocity,
                        note_on.time, note_off.time)
                       for note_on, note_off in note_pairs]
    elif mode == "single_melody_harmony":
        if len(chords) < len(note_pairs):
            print("ERROR: Number of chords must match number of melody notes!")
            return note_events

        note_events = [(note_on.note, note_on.velocity,
                        note_on.time, note_off.time,
                        chords[i])
                       for i, (note_on, note_off) in enumerate(note_pairs)]
    elif mode == "drums":
        for note_pair in note_pairs:
            note_ons = note_pair[0]
            note_offs = note_pair[1]
            
            note_on_list = [(note_on.note, note_on.velocity) for note_on in note_ons]
            note_on_list = sorted(note_on_list, key=lambda x: x[0])
            note_on_time = note_ons[0].time
            note_off_time = note_offs[0].time
            # print([note_off.time for note_off in note_offs])  # Should all be equal

            note_event = (note_on_list, note_on_time, note_off_time)
            note_events.append(note_event)

    return note_events

In [19]:
note_events = create_note_events(note_pairs, mode="drums")
# note_set = midi.create_note_set(note_events)
# len(note_set)

In [20]:
note_events[0]

([(51, 60)], 0, 48)

### Fixing polyphony, take 2

Let's try this again. This time we'll just try to get the soonest note off for each note on. And since the time delta for certain note ons will be 0, this should achieve the same effect as manually handling polyphony.

Functions to update:
- midi.extract_note_pairs

In [21]:
def extract_note_pairs(track):
    """Extract note on/off pairs from a MIDI track."""
    notes = midi.extract_note_messages(track)
    note_pairs = []
    
    # TODO: Test that this method still works for the old modes
    # `single_melody` and `single_melody_harmony`
    for i, note in enumerate(notes):
        if note.type == "note_on":
            # Register our note on
            note_on = note
            
            # Find the earliest subsequent note off for this note on,
            # and then create a note pair out of it
            for other_note in notes[i:]:
                if other_note.type == "note_off" \
                and other_note.note == note.note \
                and other_note.time != 0:
                    note_off = other_note
                    note_pairs.append((note_on, note_off))
                    break
    
    # Old method where we don't look beyond the note immediately following
    # the note on event
    # note_pairs = [(notes[i], notes[i + 1]) for i, _ in enumerate(notes[:-1])
    #               if notes[i].type == "note_on" and
    #               notes[i + 1].type == "note_off" and
    #               notes[i].note == notes[i + 1].note]

    return note_pairs

In [22]:
midi_track = midi.load_melody_from_file(input_filepath)
note_pairs = extract_note_pairs(midi_track)
note_pairs = midi.normalize_velocities(note_pairs, interval=10)
len(note_pairs)

1024

In [23]:
note_pairs[:10]

[(<message note_on channel=0 note=51 velocity=60 time=0>,
  <message note_off channel=0 note=51 velocity=0 time=48>),
 (<message note_on channel=0 note=38 velocity=50 time=16>,
  <message note_off channel=0 note=38 velocity=0 time=16>),
 (<message note_on channel=0 note=44 velocity=70 time=32>,
  <message note_off channel=0 note=44 velocity=0 time=32>),
 (<message note_on channel=0 note=51 velocity=70 time=0>,
  <message note_off channel=0 note=51 velocity=0 time=32>),
 (<message note_on channel=0 note=51 velocity=70 time=16>,
  <message note_off channel=0 note=51 velocity=0 time=32>),
 (<message note_on channel=0 note=38 velocity=60 time=32>,
  <message note_off channel=0 note=38 velocity=0 time=16>),
 (<message note_on channel=0 note=51 velocity=60 time=0>,
  <message note_off channel=0 note=51 velocity=0 time=32>),
 (<message note_on channel=0 note=36 velocity=60 time=96>,
  <message note_off channel=0 note=36 velocity=0 time=48>),
 (<message note_on channel=0 note=44 velocity=60 ti

In [24]:
# Number of notes that immediately follow the previous note
len([1 for note_pair in note_pairs if note_pair[0].time == 0])

411

In [25]:
note_events = midi.create_note_events(note_pairs, mode="single_melody")
note_set = midi.create_note_set(note_events)
len(note_set)

248

OK brb let's add this to the module!

### Training and generating, with polyphony handling (supposedly) added

We'll keep updating/adding previous sections (in addition to the actual `jazzaiexperiments` module) until this one does what we want

In [26]:
# import importlib
# importlib.reload(jazzaiexperiments)

In [27]:
# %load_ext autoreload
# %autoreload 2

^ these didn't quite work unfortunately, so we'll just restart the kernel

In [28]:
jazzaiexperiments.midi.extract_note_pairs??

In [29]:
jazzaiexperiments.midi.create_note_events??

In [30]:
model, note_events, input_filepath = jazzaiexperiments.lstm.train_on_midi_input(tune_name,
                                                                                mode="drums",
                                                                                input_filepath=input_filepath,
                                                                                num_epochs=0)

Created 1024 note events from ../data/midi/mine/billyboy/PhillyJoeJones_BillyBoy_Solo_Processed77.mid using mode drums
Formatted note data (1014 seqs of length 10, 248 unique notes)
Created model
No weights loaded (to load weights, specify a `weights_filepath`)
Epoch 1/1
1014/1014 [==============================] - 6s - loss: 4.9361     
Trained model over 1 epochs


In [31]:
model = jazzaiexperiments.lstm.load_model_weights(model, "../data/models/weights_billyboy_solo_20170706181343180726_97_0.3724.hdf5")
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 10, 256)           264192    
_________________________________________________________________
dropout_1 (Dropout)          (None, 10, 256)           0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 256)               525312    
_________________________________________________________________
dropout_2 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 248)               63736     
Total params: 853,240
Trainable params: 853,240
Non-trainable params: 0
_________________________________________________________________


In [35]:
notes_out = jazzaiexperiments.lstm.generate_midi_output(model, note_events,
                                                        mode="drums",
                                                        num_notes_to_generate=100,
                                                        time_multiplier=5,
                                                        random_seed=True,
                                                        add_seed_to_output=True,
                                                        tune_name=tune_name,
                                                        midi_source_filepath=input_filepath,
                                                        data_dir="../data/output")
notes_out[:20]

Constructed input sequence: [229, 170, 231, 103, 231, 231, 63, 126, 172, 238]
Generated 100 notes
Wrote to MIDI file at ../data/output/out_billyboy_solo_20170706203330969893.mid


([(51, 50, 0, 32),
  (44, 60, 96, 48),
  (51, 60, 0, 32),
  (38, 60, 16, 16),
  (51, 60, 0, 32),
  (51, 60, 0, 32),
  (38, 50, 16, 32),
  (38, 70, 0, 48),
  (44, 70, 0, 32),
  (51, 70, 0, 32),
  (51, 60, 16, 32),
  (51, 60, 0, 48),
  (38, 50, 16, 16),
  (44, 70, 32, 32),
  (51, 70, 0, 32),
  (51, 60, 16, 32),
  (51, 60, 0, 48),
  (38, 50, 16, 16),
  (44, 70, 32, 32),
  (51, 70, 0, 32),
  (38, 50, 16, 32),
  (38, 60, 1, 31),
  (38, 70, 16, 32),
  (38, 60, 0, 31),
  (38, 60, 1, 31),
  (38, 60, 1, 32),
  (36, 50, 32, 96),
  (38, 60, 64, 32),
  (38, 60, 0, 31),
  (38, 60, 1, 31),
  (38, 60, 1, 31),
  (38, 60, 0, 31),
  (38, 60, 1, 32),
  (38, 60, 1, 31),
  (38, 60, 1, 31),
  (38, 60, 0, 31),
  (47, 80, 32, 31),
  (38, 50, 1, 31),
  (38, 60, 1, 31),
  (38, 60, 1, 16),
  (51, 70, 32, 32),
  (44, 60, 96, 48),
  (51, 60, 0, 32),
  (38, 60, 16, 16),
  (51, 60, 0, 32),
  (51, 70, 0, 32),
  (36, 60, 80, 48),
  (44, 60, 0, 32),
  (46, 60, 0, 24),
  (51, 60, 16, 32),
  (51, 70, 0, 48),
  (38, 50, 1

In [33]:
jazzaiexperiments.midi.write_file(note_events, os.path.join("../data/output/", "{}_original.mid".format(tune_name)),
                                  mode="single_melody",
                                  time_multiplier=5,
                                  midi_source_filepath=input_filepath)


'../data/output/billyboy_solo_original.mid'

OK, it's getting closer. But the output method needs to be adjusted so that the `note_off` of a note pair only happens after all subsequent `note_on`s with `time==0` happen. Otherwise it just sounds weirdly staggered

### Fixing MIDI timing

So that we can FINALLY eliminate the need for the `time_multiplier`..

In [34]:
# TODO